# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
#Import CSV
weather = pd.read_csv("../WeatherPy/Outputs/Weather_Data")
weather

,City,County,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Bluff,NZ,-46.6000,168.3333,12.24,92,100,12.91
1,Ushuaia,AR,-54.8000,-68.3000,20.00,52,0,3.68
2,Tumalim,PH,14.0801,120.7231,25.69,89,75,1.54
3,Rikitea,PF,-23.1203,-134.9692,25.60,75,50,4.06
4,Lashma,RU,54.9321,41.1438,-13.63,94,100,3.11
...,...,...,...,...,...,...,...,...
559,Havre-St-Pierre,CA,50.2334,-63.5986,-17.00,55,1,6.69
560,Lipari,IT,38.4674,14.9540,20.56,44,66,0.45
561,Sárvár,HU,47.2539,16.9352,2.21,100,90,2.06
562,Abu Samrah,SY,35.3029,37.1841,16.13,52,0,6.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [35]:
# Store Latitude and Longitude into  locations and make float
locations = weather[["Latitude", "Longitude"]].astype(float)

#Ensure humidity is float typle
humid = weather["Humidity"].astype(float)

In [40]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
#Slim down df to no cloudiness, wind less than 10 mph, and temps between 18-26 degrees C (65-80 degree F)
#No cloudiness
no_clouds = weather.loc[weather['Cloudiness'] == 0]
no_clouds

#winds less than 10 mph
slow_wind = no_clouds.loc[no_clouds['Wind Speed'] < 10]
slow_wind

#temp between 18-26
ideal_weather = slow_wind.loc[(slow_wind['Temperature'] > 18) & (slow_wind['Temperature'] < 26)]
print(f'There are {len(ideal_weather)} cities in this dataset')
ideal_weather

There are 26 cities in this dataset


,City,County,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
1,Ushuaia,AR,-54.8000,-68.3000,20.00,52,0,3.68
12,Leeton,AU,-34.5667,146.4000,20.00,64,0,1.54
19,Vao,NC,-22.6667,167.4833,24.89,84,0,7.68
27,Teknāf,BD,20.8624,92.3058,20.16,61,0,3.62
44,Geraldton,AU,-28.7667,114.6000,19.00,55,0,9.26
54,Saint-Philippe,RE,-21.3585,55.7679,25.73,65,0,5.66
79,Allāpalli,IN,19.4167,80.0667,20.65,23,0,0.19
86,Chui,UY,-33.6971,-53.4616,24.62,72,0,3.76
98,Coyhaique,CL,-45.5752,-72.0662,21.00,52,0,2.06
125,Jalu,LY,29.0331,21.5482,24.29,16,0,3.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
